<a href="https://colab.research.google.com/github/HwangSiyeon/ESAA_homework/blob/main/3_15_%EB%AA%A8%EB%8D%B8%ED%9B%88%EB%A0%A8_%EC%84%B8%EC%85%98_%EC%97%B0%EC%8A%B5%EB%AC%B8%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **| 모델 훈련 연습 문제**
___
- 출처 : 핸즈온 머신러닝 Ch04 연습문제 1, 5, 9, 10
- 개념 문제의 경우 텍스트 셀을 추가하여 정답을 적어주세요.

### **1. 수백만 개의 특성을 가진 훈련 세트에서는 어떤 선형 회귀 알고리즘을 사용할 수 있을까요?**
___


확률적 경사 하강법이나 미니배치 경사 하강법 훈련 세트가 메모리 크기에 맞으면 배치 경사 하강법도 가능하다. 하지만 정규 방정식이나 SVD 방법은 계산 복잡도가 특성 개수에 따라 매우 빠르게 증가하기 때문에 사용할 수 없다.

### **2. 배치 경사 하강법을 사용하고 에포크마다 검증 오차를 그래프로 나타내봤습니다. 검증 오차가 일정하게 상승되고 있다면 어떤 일이 일어나고 있는 걸까요? 이 문제를 어떻게 해결할 수 있나요?**
___

학습률이 너무 크면 모델이 발산할 수 있다. 학습률을 줄이면 된다.

### **3. 릿지 회귀를 사용했을 때 훈련 오차가 검증 오차가 거의 비슷하고 둘 다 높았습니다. 이 모델에는 높은 편향이 문제인가요, 아니면 높은 분산이 문제인가요? 규제 하이퍼파라미터 $\alpha$를 증가시켜야 할까요 아니면 줄여야 할까요?**
___

모델의 과소적합이 발생한다. 높은 편향이 문제이다. 알파값을 감소시켜야 한다.

### **4. 다음과 같이 사용해야 하는 이유는?**
___
- 평범한 선형 회귀(즉, 아무런 규제가 없는 모델) 대신 릿지 회귀
- 릿지 회귀 대신 라쏘 회귀
- 라쏘 회귀 대신 엘라스틱넷

- 첫번째 경우는 규제를 강화하여 오버피팅을 방지하기 위해서이다
- 두번째 경우는 중요한 특징은 덜 중요한 특성의 가중치를 제거하려고 할 때 사용한다
- 세번째 경우에서 엘라스틱넷은 릿지회귀와 랏쏘회귀를 절충한 모델으로,

### **추가) 조기 종료를 사용한 배치 경사 하강법으로 iris 데이터를 활용해 소프트맥스 회귀를 구현해보세요(사이킷런은 사용하지 마세요)**

---



In [ ]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

# Load iris dataset
iris = load_iris()
X, y = iris.data, iris.target

# Normalize features
X = (X - X.mean(axis=0)) / X.std(axis=0)

# One-hot encode the target variable
encoder = OneHotEncoder(categories='auto')
y_onehot = encoder.fit_transform(y.reshape(-1, 1)).toarray()

# Add bias term to features
X = np.c_[np.ones((X.shape[0], 1)), X]

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y_onehot, test_size=0.2, random_state=42)

# Softmax function
def softmax(z):
    exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))
    return exp_z / np.sum(exp_z, axis=1, keepdims=True)

# Cross-entropy loss function
def cross_entropy_loss(y_true, y_pred):
    return -np.mean(np.sum(y_true * np.log(y_pred), axis=1))

# Gradient of cross-entropy loss function
def grad_cross_entropy_loss(y_true, y_pred):
    return y_pred - y_true

# Batch gradient descent with early stopping
def batch_gradient_descent(X, y, learning_rate, epochs, tolerance):
    np.random.seed(42)
    theta = np.random.randn(X.shape[1], y.shape[1])
    best_loss = float('inf')
    best_epoch = 0
    for epoch in range(epochs):
        logits = np.dot(X, theta)
        y_pred = softmax(logits)
        loss = cross_entropy_loss(y, y_pred)
        if epoch % 100 == 0:
            print(f"Epoch {epoch}: Loss {loss}")
        if loss < best_loss - tolerance:
            best_loss = loss
            best_epoch = epoch
        elif epoch - best_epoch > 1000:
            print("Early stopping.")
            break
        gradient = np.dot(X.T, grad_cross_entropy_loss(y, y_pred)) / len(X)
        theta -= learning_rate * gradient
    return theta

# Train softmax regression model
learning_rate = 0.01
epochs = 10000
tolerance = 1e-6
theta = batch_gradient_descent(X_train, y_train, learning_rate, epochs, tolerance)

# Make predictions on test set
logits_test = np.dot(X_test, theta)
y_pred_test = softmax(logits_test)
predicted_labels = np.argmax(y_pred_test, axis=1)

# Calculate accuracy
accuracy = np.mean(predicted_labels == np.argmax(y_test, axis=1))
print(f"Accuracy on test set: {accuracy}")
